# 📈 Trading Strategy Integration

## Convert Hyperparameter Optimizations to Live Trading Strategies

This notebook bridges the gap between hyperparameter optimization results and actual trading implementation:

- **Model Deployment**: Load optimized models for live trading
- **Signal Generation**: Convert model predictions to trading signals
- **Risk Management**: Implement position sizing and risk controls
- **Backtesting**: Validate strategies on historical data
- **Live Trading**: Deploy strategies for real-time execution
- **Performance Monitoring**: Track strategy performance metrics

In [1]:
# Trading Strategy Integration Framework
import os
import sys
import json
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from pathlib import Path
from typing import Dict, List, Tuple, Optional, Any, Union
import warnings
from dataclasses import dataclass, field

warnings.filterwarnings('ignore')

# ML and trading libraries
import tensorflow as tf
from tensorflow.keras.models import load_model, Sequential
from tensorflow.keras.layers import Conv1D, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.optimizers import Adam

from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report

# Technical analysis
try:
    import ta
    from ta.volatility import BollingerBands, AverageTrueRange
    from ta.trend import MACD, ADXIndicator
    from ta.momentum import RSIIndicator, StochasticOscillator
    print("✅ Technical analysis library available")
except ImportError:
    print("Installing technical analysis library...")
    import subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "ta"])
    import ta
    from ta.volatility import BollingerBands, AverageTrueRange
    from ta.trend import MACD, ADXIndicator
    from ta.momentum import RSIIndicator, StochasticOscillator
    print("✅ Technical analysis library installed")

# Configuration
SYMBOLS = ['EURUSD', 'GBPUSD', 'USDJPY', 'AUDUSD', 'USDCAD', 'EURJPY', 'GBPJPY']
DATA_PATH = "data"
RESULTS_PATH = "optimization_results"
MODELS_PATH = "exported_models"
STRATEGIES_PATH = "trading_strategies"

# Create directories
for path in [DATA_PATH, RESULTS_PATH, MODELS_PATH, STRATEGIES_PATH]:
    Path(path).mkdir(exist_ok=True)

# Trading configuration
TRADING_CONFIG = {
    'initial_capital': 10000,
    'max_position_size': 0.02,  # 2% of capital per trade
    'stop_loss_pct': 0.02,      # 2% stop loss
    'take_profit_pct': 0.04,    # 4% take profit
    'max_daily_trades': 5,
    'risk_free_rate': 0.02,     # 2% annual risk-free rate
    'slippage': 0.0001,         # 1 pip slippage
    'commission': 0.00002       # 0.2 pip commission
}

print(f"🚀 Trading Strategy Integration System Initialized")
print(f"Target symbols: {SYMBOLS}")
print(f"Trading config: {TRADING_CONFIG}")

# Suppress TensorFlow warnings
tf.get_logger().setLevel('ERROR')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

2025-06-12 22:59:39.058535: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749733179.073499   14410 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749733179.077915   14410 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749733179.089019   14410 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749733179.089043   14410 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1749733179.089044   14410 computation_placer.cc:177] computation placer alr

✅ Technical analysis library available
🚀 Trading Strategy Integration System Initialized
Target symbols: ['EURUSD', 'GBPUSD', 'USDJPY', 'AUDUSD', 'USDCAD', 'EURJPY', 'GBPJPY']
Trading config: {'initial_capital': 10000, 'max_position_size': 0.02, 'stop_loss_pct': 0.02, 'take_profit_pct': 0.04, 'max_daily_trades': 5, 'risk_free_rate': 0.02, 'slippage': 0.0001, 'commission': 2e-05}


In [2]:
# Data Classes for Trading Strategy
@dataclass
class TradingSignal:
    """Trading signal with metadata"""
    timestamp: datetime
    symbol: str
    signal: int  # 1 for buy, -1 for sell, 0 for hold
    confidence: float
    price: float
    stop_loss: Optional[float] = None
    take_profit: Optional[float] = None
    position_size: Optional[float] = None
    metadata: Dict[str, Any] = field(default_factory=dict)

@dataclass
class TradeExecution:
    """Trade execution record"""
    trade_id: str
    symbol: str
    entry_time: datetime
    entry_price: float
    exit_time: Optional[datetime] = None
    exit_price: Optional[float] = None
    position_size: float = 0.0
    direction: int = 0  # 1 for long, -1 for short
    pnl: float = 0.0
    pnl_pct: float = 0.0
    status: str = "open"  # open, closed, stopped_out
    stop_loss: Optional[float] = None
    take_profit: Optional[float] = None
    commission: float = 0.0
    slippage: float = 0.0

@dataclass
class StrategyPerformance:
    """Strategy performance metrics"""
    symbol: str
    total_trades: int
    winning_trades: int
    losing_trades: int
    win_rate: float
    total_return: float
    annual_return: float
    sharpe_ratio: float
    max_drawdown: float
    profit_factor: float
    avg_trade: float
    avg_win: float
    avg_loss: float
    largest_win: float
    largest_loss: float

print("✅ Trading data classes defined")

✅ Trading data classes defined


In [3]:
class OptimizedModelLoader:
    """Load and manage optimized models for trading"""
    
    def __init__(self, results_path: str = RESULTS_PATH, models_path: str = MODELS_PATH):
        self.results_path = Path(results_path)
        self.models_path = Path(models_path)
        self.loaded_models: Dict[str, Any] = {}
        self.model_configs: Dict[str, Dict[str, Any]] = {}
        self.scalers: Dict[str, Any] = {}
        
    def load_best_parameters(self, symbol: str) -> Optional[Dict[str, Any]]:
        """Load the best optimization parameters for a symbol"""
        param_files = list(self.results_path.glob(f"best_params_{symbol}_*.json"))
        
        if not param_files:
            print(f"❌ No optimization results found for {symbol}")
            return None
        
        # Get the most recent optimization result
        latest_file = max(param_files, key=lambda x: x.stat().st_mtime)
        
        try:
            with open(latest_file, 'r') as f:
                data = json.load(f)
            
            print(f"✅ Loaded parameters for {symbol} from {latest_file.name}")
            print(f"   Objective value: {data.get('objective_value', 0):.6f}")
            print(f"   Accuracy: {data.get('mean_accuracy', 0):.4f}")
            
            return data
            
        except Exception as e:
            print(f"❌ Failed to load parameters for {symbol}: {e}")
            return None
    
    def build_optimized_model(self, symbol: str, input_shape: Tuple[int, int]) -> Optional[tf.keras.Model]:
        """Build a model using optimized hyperparameters"""
        config_data = self.load_best_parameters(symbol)
        
        if not config_data or 'best_params' not in config_data:
            print(f"❌ No valid parameters found for {symbol}")
            return None
        
        params = config_data['best_params']
        
        try:
            # Build model with optimized parameters
            model = Sequential()
            
            # First Conv1D layer
            model.add(Conv1D(
                filters=params.get('conv1d_filters_1', 64),
                kernel_size=params.get('conv1d_kernel_size', 3),
                activation='relu',
                input_shape=input_shape,
                kernel_regularizer=l1_l2(
                    l1=params.get('l1_reg', 1e-4),
                    l2=params.get('l2_reg', 1e-4)
                )
            ))
            
            if params.get('batch_normalization', True):
                model.add(BatchNormalization())
            
            model.add(Dropout(params.get('dropout_rate', 0.2)))
            
            # Second Conv1D layer
            model.add(Conv1D(
                filters=params.get('conv1d_filters_2', 32),
                kernel_size=params.get('conv1d_kernel_size', 3),
                activation='relu',
                kernel_regularizer=l1_l2(
                    l1=params.get('l1_reg', 1e-4),
                    l2=params.get('l2_reg', 1e-4)
                )
            ))
            
            if params.get('batch_normalization', True):
                model.add(BatchNormalization())
            
            model.add(Dropout(params.get('dropout_rate', 0.2)))
            
            # LSTM layer
            model.add(LSTM(
                units=params.get('lstm_units', 50),
                return_sequences=params.get('lstm_return_sequences', False),
                kernel_regularizer=l1_l2(
                    l1=params.get('l1_reg', 1e-4),
                    l2=params.get('l2_reg', 1e-4)
                )
            ))
            
            model.add(Dropout(params.get('dropout_rate', 0.2)))
            
            # Dense layers
            num_dense_layers = params.get('num_dense_layers', 1)
            for i in range(num_dense_layers):
                units = params.get('dense_units', 32) // (i + 1)
                if units < 5:
                    break
                
                model.add(Dense(
                    units=units,
                    activation='relu',
                    kernel_regularizer=l1_l2(
                        l1=params.get('l1_reg', 1e-4),
                        l2=params.get('l2_reg', 1e-4)
                    )
                ))
                
                if i < num_dense_layers - 1:
                    model.add(Dropout(params.get('dropout_rate', 0.2) * 0.5))
            
            # Output layer
            model.add(Dense(1, activation='sigmoid'))
            
            # Compile model
            model.compile(
                optimizer=Adam(learning_rate=params.get('learning_rate', 0.001)),
                loss='binary_crossentropy',
                metrics=['accuracy']
            )
            
            # Store configuration
            self.model_configs[symbol] = {
                'params': params,
                'objective_value': config_data.get('objective_value', 0),
                'mean_accuracy': config_data.get('mean_accuracy', 0),
                'timestamp': config_data.get('timestamp', 'unknown')
            }
            
            self.loaded_models[symbol] = model
            
            print(f"✅ Built optimized model for {symbol}")
            print(f"   Model parameters: {model.count_params():,}")
            
            return model
            
        except Exception as e:
            print(f"❌ Failed to build model for {symbol}: {e}")
            return None
    
    def save_trained_model(self, symbol: str, model: tf.keras.Model, scaler: Any = None):
        """Save a trained model for later use"""
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        
        # Save model
        model_file = self.models_path / f"trained_model_{symbol}_{timestamp}.h5"
        model.save(model_file)
        
        # Save scaler if provided
        if scaler is not None:
            scaler_file = self.models_path / f"scaler_{symbol}_{timestamp}.pkl"
            with open(scaler_file, 'wb') as f:
                pickle.dump(scaler, f)
            self.scalers[symbol] = scaler
        
        # Save metadata
        metadata = {
            'symbol': symbol,
            'timestamp': timestamp,
            'model_file': str(model_file),
            'scaler_file': str(scaler_file) if scaler else None,
            'config': self.model_configs.get(symbol, {})
        }
        
        metadata_file = self.models_path / f"model_metadata_{symbol}_{timestamp}.json"
        with open(metadata_file, 'w') as f:
            json.dump(metadata, f, indent=2)
        
        print(f"✅ Saved trained model for {symbol}:")
        print(f"   Model: {model_file}")
        print(f"   Scaler: {scaler_file if scaler else 'None'}")
        print(f"   Metadata: {metadata_file}")
    
    def load_trained_model(self, symbol: str) -> Tuple[Optional[tf.keras.Model], Optional[Any]]:
        """Load the most recent trained model for a symbol"""
        model_files = list(self.models_path.glob(f"trained_model_{symbol}_*.h5"))
        
        if not model_files:
            print(f"❌ No trained models found for {symbol}")
            return None, None
        
        # Get the most recent model
        latest_model_file = max(model_files, key=lambda x: x.stat().st_mtime)
        
        try:
            # Load model
            model = load_model(latest_model_file)
            
            # Find corresponding scaler
            timestamp = latest_model_file.stem.split('_')[-1]
            scaler_file = self.models_path / f"scaler_{symbol}_{timestamp}.pkl"
            
            scaler = None
            if scaler_file.exists():
                with open(scaler_file, 'rb') as f:
                    scaler = pickle.load(f)
            
            self.loaded_models[symbol] = model
            if scaler:
                self.scalers[symbol] = scaler
            
            print(f"✅ Loaded trained model for {symbol}")
            print(f"   Model: {latest_model_file}")
            print(f"   Scaler: {'Found' if scaler else 'Not found'}")
            
            return model, scaler
            
        except Exception as e:
            print(f"❌ Failed to load trained model for {symbol}: {e}")
            return None, None

# Initialize model loader
model_loader = OptimizedModelLoader()
print("✅ OptimizedModelLoader initialized")

✅ OptimizedModelLoader initialized


In [4]:
class DataPipeline:
    """Data pipeline for feature engineering and preprocessing"""
    
    def __init__(self, data_path: str = DATA_PATH):
        self.data_path = Path(data_path)
        self.cached_data: Dict[str, pd.DataFrame] = {}
        self.cached_features: Dict[str, pd.DataFrame] = {}
        
    def load_symbol_data(self, symbol: str) -> Optional[pd.DataFrame]:
        """Load price data for a symbol"""
        if symbol in self.cached_data:
            return self.cached_data[symbol]
        
        # Try different file formats
        file_patterns = [
            f"metatrader_{symbol}.parquet",
            f"metatrader_{symbol}.h5",
            f"{symbol}.parquet",
            f"{symbol}.h5",
            f"{symbol}.csv"
        ]
        
        for pattern in file_patterns:
            file_path = self.data_path / pattern
            if file_path.exists():
                try:
                    if pattern.endswith('.parquet'):
                        df = pd.read_parquet(file_path)
                    elif pattern.endswith('.h5'):
                        df = pd.read_hdf(file_path, key='data')
                    elif pattern.endswith('.csv'):
                        df = pd.read_csv(file_path, index_col=0, parse_dates=True)
                    else:
                        continue
                    
                    # Standardize columns
                    df.columns = [col.lower() for col in df.columns]
                    
                    # Ensure datetime index
                    if not isinstance(df.index, pd.DatetimeIndex):
                        df.index = pd.to_datetime(df.index)
                    
                    # Sort by index
                    df = df.sort_index()
                    
                    self.cached_data[symbol] = df
                    print(f"✅ Loaded {symbol}: {len(df)} rows from {file_path.name}")
                    return df
                    
                except Exception as e:
                    print(f"⚠️ Failed to load {file_path}: {e}")
                    continue
        
        print(f"❌ No data file found for {symbol} in {self.data_path}")
        return None
    
    def calculate_technical_indicators(self, df: pd.DataFrame) -> pd.DataFrame:
        """Calculate technical indicators for trading"""
        indicators = pd.DataFrame(index=df.index)
        
        # Price data
        close = df['close']
        high = df['high'] if 'high' in df.columns else close
        low = df['low'] if 'low' in df.columns else close
        volume = df.get('tick_volume', pd.Series(index=df.index, data=1000))
        
        try:
            # Price-based features
            indicators['returns'] = close.pct_change()
            indicators['log_returns'] = np.log(close / close.shift(1))
            
            # Moving averages
            for period in [5, 10, 20, 50]:
                indicators[f'sma_{period}'] = close.rolling(period).mean()
                indicators[f'ema_{period}'] = close.ewm(span=period).mean()
                indicators[f'price_to_sma_{period}'] = close / indicators[f'sma_{period}']
            
            # Volatility
            for period in [5, 10, 20]:
                indicators[f'volatility_{period}'] = close.rolling(period).std()
                indicators[f'volatility_{period}_norm'] = indicators[f'volatility_{period}'] / close
            
            # RSI
            indicators['rsi'] = RSIIndicator(close=close, window=14).rsi()
            indicators['rsi_fast'] = RSIIndicator(close=close, window=7).rsi()
            indicators['rsi_oversold'] = (indicators['rsi'] < 30).astype(int)
            indicators['rsi_overbought'] = (indicators['rsi'] > 70).astype(int)
            
            # MACD
            macd = MACD(close=close)
            indicators['macd'] = macd.macd()
            indicators['macd_signal'] = macd.macd_signal()
            indicators['macd_histogram'] = macd.macd_diff()
            indicators['macd_bullish'] = (indicators['macd'] > indicators['macd_signal']).astype(int)
            
            # Bollinger Bands
            bb = BollingerBands(close=close, window=20, window_dev=2)
            indicators['bb_upper'] = bb.bollinger_hband()
            indicators['bb_lower'] = bb.bollinger_lband()
            indicators['bb_middle'] = bb.bollinger_mavg()
            indicators['bb_width'] = bb.bollinger_wband()
            indicators['bb_position'] = (close - indicators['bb_lower']) / (indicators['bb_upper'] - indicators['bb_lower'])
            
            # ATR
            indicators['atr'] = AverageTrueRange(high=high, low=low, close=close).average_true_range()
            indicators['atr_norm'] = indicators['atr'] / close
            
            # Stochastic
            stoch = StochasticOscillator(high=high, low=low, close=close)
            indicators['stoch_k'] = stoch.stoch()
            indicators['stoch_d'] = stoch.stoch_signal()
            
            # Price momentum
            for period in [1, 3, 5, 10]:
                indicators[f'momentum_{period}'] = close.pct_change(period)
                indicators[f'price_change_{period}'] = (close > close.shift(period)).astype(int)
            
            # Volume indicators (if available)
            if 'tick_volume' in df.columns:
                indicators['volume_sma_20'] = volume.rolling(20).mean()
                indicators['volume_ratio'] = volume / indicators['volume_sma_20']
            
            # Time-based features
            indicators['hour'] = indicators.index.hour
            indicators['day_of_week'] = indicators.index.dayofweek
            indicators['month'] = indicators.index.month
            indicators['is_weekend'] = (indicators.index.dayofweek >= 5).astype(int)
            
            # Market session
            def get_session(hour):
                if 0 <= hour < 8:
                    return 1  # Asian
                elif 8 <= hour < 16:
                    return 2  # European
                else:
                    return 3  # US
            
            indicators['market_session'] = indicators['hour'].apply(get_session)
            
        except Exception as e:
            print(f"⚠️ Error calculating some indicators: {e}")
        
        # Clean data
        indicators = indicators.fillna(method='ffill').fillna(method='bfill').fillna(0)
        indicators = indicators.replace([np.inf, -np.inf], 0)
        
        return indicators
    
    def create_trading_features(self, symbol: str, lookback_window: int = 20) -> Optional[pd.DataFrame]:
        """Create features ready for trading model"""
        if symbol in self.cached_features:
            return self.cached_features[symbol]
        
        # Load price data
        price_data = self.load_symbol_data(symbol)
        if price_data is None:
            return None
        
        # Calculate indicators
        features = self.calculate_technical_indicators(price_data)
        
        # Add lookback window information
        features['lookback_window'] = lookback_window
        
        # Cache features
        self.cached_features[symbol] = features
        
        print(f"✅ Created {features.shape[1]} features for {symbol}")
        return features
    
    def prepare_model_input(self, features: pd.DataFrame, lookback_window: int = 20, end_date: Optional[datetime] = None) -> Tuple[np.ndarray, pd.Index]:
        """Prepare features for model input"""
        if end_date:
            features = features[features.index <= end_date]
        
        # Select numerical features only
        numerical_features = features.select_dtypes(include=[np.number])
        
        # Create sequences
        X = []
        valid_indices = []
        
        for i in range(lookback_window, len(numerical_features)):
            X.append(numerical_features.iloc[i-lookback_window:i].values)
            valid_indices.append(numerical_features.index[i])
        
        if len(X) == 0:
            return np.array([]), pd.Index([])
        
        X = np.array(X)
        
        return X, pd.Index(valid_indices)

# Initialize data pipeline
data_pipeline = DataPipeline()
print("✅ DataPipeline initialized")

✅ DataPipeline initialized


In [5]:
class TradingSignalGenerator:
    """Generate trading signals from model predictions"""
    
    def __init__(self, model_loader: OptimizedModelLoader, data_pipeline: DataPipeline):
        self.model_loader = model_loader
        self.data_pipeline = data_pipeline
        self.signal_history: Dict[str, List[TradingSignal]] = {}
        
    def generate_signals(
        self, 
        symbol: str, 
        current_time: datetime,
        confidence_threshold_high: float = 0.6,
        confidence_threshold_low: float = 0.4
    ) -> Optional[TradingSignal]:
        """Generate trading signal for a symbol at current time"""
        
        # Get or load model
        if symbol not in self.model_loader.loaded_models:
            model, scaler = self.model_loader.load_trained_model(symbol)
            if model is None:
                print(f"❌ No trained model available for {symbol}")
                return None
        else:
            model = self.model_loader.loaded_models[symbol]
            scaler = self.model_loader.scalers.get(symbol)
        
        # Get model configuration
        config = self.model_loader.model_configs.get(symbol, {})
        params = config.get('params', {})
        lookback_window = params.get('lookback_window', 20)
        
        # Prepare features
        features = self.data_pipeline.create_trading_features(symbol, lookback_window)
        if features is None:
            print(f"❌ No features available for {symbol}")
            return None
        
        # Get current price
        price_data = self.data_pipeline.load_symbol_data(symbol)
        if price_data is None:
            return None
        
        # Find the most recent price data
        recent_data = price_data[price_data.index <= current_time]
        if len(recent_data) == 0:
            return None
        
        current_price = recent_data['close'].iloc[-1]
        
        # Prepare model input
        X, indices = self.data_pipeline.prepare_model_input(
            features, lookback_window, current_time
        )
        
        if len(X) == 0:
            return None
        
        # Use the most recent sequence
        X_current = X[-1:]
        
        # Scale features if scaler is available
        if scaler:
            # Reshape for scaling
            X_flat = X_current.reshape(X_current.shape[0], -1)
            X_scaled = scaler.transform(X_flat)
            X_current = X_scaled.reshape(X_current.shape)
        
        try:
            # Make prediction
            prediction = model.predict(X_current, verbose=0)[0][0]
            
            # Generate signal based on confidence thresholds
            if prediction > confidence_threshold_high:
                signal = 1  # Buy
            elif prediction < confidence_threshold_low:
                signal = -1  # Sell
            else:
                signal = 0  # Hold
            
            # Calculate position sizing and risk management
            atr = features['atr'].iloc[-1] if 'atr' in features.columns else current_price * 0.01
            
            # Stop loss and take profit based on ATR
            if signal == 1:  # Buy
                stop_loss = current_price - (2 * atr)
                take_profit = current_price + (3 * atr)
            elif signal == -1:  # Sell
                stop_loss = current_price + (2 * atr)
                take_profit = current_price - (3 * atr)
            else:
                stop_loss = None
                take_profit = None
            
            # Create trading signal
            trading_signal = TradingSignal(
                timestamp=current_time,
                symbol=symbol,
                signal=signal,
                confidence=float(prediction),
                price=current_price,
                stop_loss=stop_loss,
                take_profit=take_profit,
                position_size=TRADING_CONFIG['max_position_size'],
                metadata={
                    'model_config': config,
                    'atr': atr,
                    'lookback_window': lookback_window,
                    'features_used': list(features.columns)
                }
            )
            
            # Store signal history
            if symbol not in self.signal_history:
                self.signal_history[symbol] = []
            self.signal_history[symbol].append(trading_signal)
            
            return trading_signal
            
        except Exception as e:
            print(f"❌ Error generating signal for {symbol}: {e}")
            return None
    
    def get_signal_summary(self, symbol: str, days: int = 7) -> Dict[str, Any]:
        """Get summary of recent signals for a symbol"""
        if symbol not in self.signal_history:
            return {}
        
        cutoff_date = datetime.now() - timedelta(days=days)
        recent_signals = [
            s for s in self.signal_history[symbol] 
            if s.timestamp >= cutoff_date
        ]
        
        if not recent_signals:
            return {}
        
        total_signals = len(recent_signals)
        buy_signals = len([s for s in recent_signals if s.signal == 1])
        sell_signals = len([s for s in recent_signals if s.signal == -1])
        hold_signals = len([s for s in recent_signals if s.signal == 0])
        
        avg_confidence = np.mean([s.confidence for s in recent_signals])
        
        return {
            'symbol': symbol,
            'period_days': days,
            'total_signals': total_signals,
            'buy_signals': buy_signals,
            'sell_signals': sell_signals,
            'hold_signals': hold_signals,
            'buy_rate': buy_signals / total_signals if total_signals > 0 else 0,
            'sell_rate': sell_signals / total_signals if total_signals > 0 else 0,
            'avg_confidence': avg_confidence,
            'latest_signal': recent_signals[-1] if recent_signals else None
        }

# Initialize signal generator
signal_generator = TradingSignalGenerator(model_loader, data_pipeline)
print("✅ TradingSignalGenerator initialized")

✅ TradingSignalGenerator initialized


In [6]:
class TradingStrategyBacktester:
    """Backtest trading strategies using optimized models"""
    
    def __init__(self, signal_generator: TradingSignalGenerator, trading_config: Dict[str, Any]):
        self.signal_generator = signal_generator
        self.config = trading_config
        self.trades: Dict[str, List[TradeExecution]] = {}
        self.portfolio_history: List[Dict[str, Any]] = []
        
    def run_backtest(
        self,
        symbol: str,
        start_date: datetime,
        end_date: datetime,
        confidence_threshold_high: float = 0.6,
        confidence_threshold_low: float = 0.4
    ) -> StrategyPerformance:
        """Run backtest for a symbol over a date range"""
        
        print(f"🔄 Running backtest for {symbol}")
        print(f"   Period: {start_date.date()} to {end_date.date()}")
        print(f"   Confidence thresholds: {confidence_threshold_low:.2f} - {confidence_threshold_high:.2f}")
        
        # Initialize portfolio
        capital = self.config['initial_capital']
        current_position = None
        trades = []
        
        # Get price data
        price_data = self.signal_generator.data_pipeline.load_symbol_data(symbol)
        if price_data is None:
            print(f"❌ No price data for {symbol}")
            return None
        
        # Filter data for backtest period
        backtest_data = price_data[
            (price_data.index >= start_date) & (price_data.index <= end_date)
        ]
        
        if len(backtest_data) == 0:
            print(f"❌ No data in backtest period for {symbol}")
            return None
        
        print(f"   Data points: {len(backtest_data)}")
        
        # Sample dates for signal generation (to avoid computational overhead)
        sample_frequency = max(1, len(backtest_data) // 1000)  # Sample up to 1000 points
        sample_dates = backtest_data.index[::sample_frequency]
        
        portfolio_values = []
        trade_count = 0
        
        for i, current_time in enumerate(sample_dates):
            if i % 100 == 0:
                print(f"   Progress: {i}/{len(sample_dates)} ({i/len(sample_dates)*100:.1f}%)")
            
            current_price = backtest_data.loc[current_time, 'close']
            
            # Generate signal
            signal = self.signal_generator.generate_signals(
                symbol, current_time, confidence_threshold_high, confidence_threshold_low
            )
            
            if signal is None:
                continue
            
            # Check if we need to close current position
            if current_position is not None:
                # Check stop loss / take profit
                should_close = False
                exit_reason = "signal"
                
                if current_position.direction == 1:  # Long position
                    if (current_position.stop_loss and current_price <= current_position.stop_loss) or \
                       (current_position.take_profit and current_price >= current_position.take_profit):
                        should_close = True
                        exit_reason = "stop_loss" if current_price <= current_position.stop_loss else "take_profit"
                elif current_position.direction == -1:  # Short position
                    if (current_position.stop_loss and current_price >= current_position.stop_loss) or \
                       (current_position.take_profit and current_price <= current_position.take_profit):
                        should_close = True
                        exit_reason = "stop_loss" if current_price >= current_position.stop_loss else "take_profit"
                
                # Close position on opposite signal or risk management
                if should_close or (signal.signal != 0 and signal.signal != current_position.direction):
                    # Close position
                    current_position.exit_time = current_time
                    current_position.exit_price = current_price
                    
                    # Calculate P&L
                    if current_position.direction == 1:  # Long
                        pnl = (current_position.exit_price - current_position.entry_price) * current_position.position_size
                    else:  # Short
                        pnl = (current_position.entry_price - current_position.exit_price) * current_position.position_size
                    
                    # Apply costs
                    commission = current_position.position_size * self.config['commission'] * 2  # Entry + exit
                    slippage = current_position.position_size * self.config['slippage'] * 2
                    
                    current_position.pnl = pnl - commission - slippage
                    current_position.pnl_pct = current_position.pnl / (current_position.entry_price * current_position.position_size)
                    current_position.commission = commission
                    current_position.slippage = slippage
                    current_position.status = "closed"
                    
                    # Update capital
                    capital += current_position.pnl
                    
                    trades.append(current_position)
                    current_position = None
            
            # Open new position if signal is strong enough
            if current_position is None and signal.signal != 0:
                trade_count += 1
                
                # Calculate position size
                position_value = capital * self.config['max_position_size']
                position_size = position_value / current_price
                
                # Create new position
                current_position = TradeExecution(
                    trade_id=f"{symbol}_{trade_count:04d}",
                    symbol=symbol,
                    entry_time=current_time,
                    entry_price=current_price,
                    position_size=position_size,
                    direction=signal.signal,
                    stop_loss=signal.stop_loss,
                    take_profit=signal.take_profit,
                    status="open"
                )
            
            # Record portfolio value
            current_value = capital
            if current_position:
                if current_position.direction == 1:
                    unrealized_pnl = (current_price - current_position.entry_price) * current_position.position_size
                else:
                    unrealized_pnl = (current_position.entry_price - current_price) * current_position.position_size
                current_value += unrealized_pnl
            
            portfolio_values.append({
                'timestamp': current_time,
                'value': current_value,
                'capital': capital,
                'position': current_position is not None
            })
        
        # Close any remaining position
        if current_position is not None:
            final_price = backtest_data['close'].iloc[-1]
            current_position.exit_time = backtest_data.index[-1]
            current_position.exit_price = final_price
            
            if current_position.direction == 1:
                pnl = (final_price - current_position.entry_price) * current_position.position_size
            else:
                pnl = (current_position.entry_price - final_price) * current_position.position_size
            
            commission = current_position.position_size * self.config['commission'] * 2
            slippage = current_position.position_size * self.config['slippage'] * 2
            
            current_position.pnl = pnl - commission - slippage
            current_position.pnl_pct = current_position.pnl / (current_position.entry_price * current_position.position_size)
            current_position.commission = commission
            current_position.slippage = slippage
            current_position.status = "closed"
            
            trades.append(current_position)
        
        # Store trades
        self.trades[symbol] = trades
        
        # Calculate performance metrics
        performance = self.calculate_performance_metrics(symbol, trades, portfolio_values)
        
        print(f"✅ Backtest completed for {symbol}")
        print(f"   Total trades: {performance.total_trades}")
        print(f"   Win rate: {performance.win_rate:.2%}")
        print(f"   Total return: {performance.total_return:.2%}")
        print(f"   Sharpe ratio: {performance.sharpe_ratio:.3f}")
        
        return performance
    
    def calculate_performance_metrics(self, symbol: str, trades: List[TradeExecution], portfolio_values: List[Dict]) -> StrategyPerformance:
        """Calculate comprehensive performance metrics"""
        
        if not trades:
            return StrategyPerformance(
                symbol=symbol, total_trades=0, winning_trades=0, losing_trades=0,
                win_rate=0, total_return=0, annual_return=0, sharpe_ratio=0,
                max_drawdown=0, profit_factor=0, avg_trade=0, avg_win=0, avg_loss=0,
                largest_win=0, largest_loss=0
            )
        
        # Basic trade statistics
        total_trades = len(trades)
        winning_trades = len([t for t in trades if t.pnl > 0])
        losing_trades = len([t for t in trades if t.pnl < 0])
        win_rate = winning_trades / total_trades if total_trades > 0 else 0
        
        # P&L statistics
        total_pnl = sum(t.pnl for t in trades)
        total_return = total_pnl / self.config['initial_capital']
        
        winning_pnl = sum(t.pnl for t in trades if t.pnl > 0)
        losing_pnl = abs(sum(t.pnl for t in trades if t.pnl < 0))
        
        profit_factor = winning_pnl / losing_pnl if losing_pnl > 0 else float('inf')
        
        avg_trade = total_pnl / total_trades if total_trades > 0 else 0
        avg_win = winning_pnl / winning_trades if winning_trades > 0 else 0
        avg_loss = -losing_pnl / losing_trades if losing_trades > 0 else 0
        
        largest_win = max([t.pnl for t in trades], default=0)
        largest_loss = min([t.pnl for t in trades], default=0)
        
        # Time-based metrics
        if len(portfolio_values) > 1:
            start_date = portfolio_values[0]['timestamp']
            end_date = portfolio_values[-1]['timestamp']
            days = (end_date - start_date).days
            annual_return = total_return * (365 / days) if days > 0 else 0
            
            # Sharpe ratio
            returns = []
            for i in range(1, len(portfolio_values)):
                prev_val = portfolio_values[i-1]['value']
                curr_val = portfolio_values[i]['value']
                returns.append((curr_val - prev_val) / prev_val if prev_val > 0 else 0)
            
            if len(returns) > 1:
                returns_std = np.std(returns)
                returns_mean = np.mean(returns)
                risk_free_daily = self.config['risk_free_rate'] / 365
                sharpe_ratio = (returns_mean - risk_free_daily) / returns_std if returns_std > 0 else 0
                sharpe_ratio *= np.sqrt(365)  # Annualized
            else:
                sharpe_ratio = 0
            
            # Maximum drawdown
            values = [pv['value'] for pv in portfolio_values]
            peak = values[0]
            max_drawdown = 0
            
            for value in values:
                if value > peak:
                    peak = value
                drawdown = (peak - value) / peak
                max_drawdown = max(max_drawdown, drawdown)
        else:
            annual_return = 0
            sharpe_ratio = 0
            max_drawdown = 0
        
        return StrategyPerformance(
            symbol=symbol,
            total_trades=total_trades,
            winning_trades=winning_trades,
            losing_trades=losing_trades,
            win_rate=win_rate,
            total_return=total_return,
            annual_return=annual_return,
            sharpe_ratio=sharpe_ratio,
            max_drawdown=max_drawdown,
            profit_factor=profit_factor,
            avg_trade=avg_trade,
            avg_win=avg_win,
            avg_loss=avg_loss,
            largest_win=largest_win,
            largest_loss=largest_loss
        )
    
    def create_performance_report(self, symbol: str) -> str:
        """Create a detailed performance report"""
        if symbol not in self.trades:
            return f"No backtest results found for {symbol}"
        
        trades = self.trades[symbol]
        performance = self.calculate_performance_metrics(symbol, trades, [])
        
        report = []
        report.append(f"# Trading Strategy Performance Report: {symbol}")
        report.append(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        report.append("\n## Overview")
        report.append(f"- Total Trades: {performance.total_trades}")
        report.append(f"- Winning Trades: {performance.winning_trades}")
        report.append(f"- Losing Trades: {performance.losing_trades}")
        report.append(f"- Win Rate: {performance.win_rate:.2%}")
        
        report.append("\n## Returns")
        report.append(f"- Total Return: {performance.total_return:.2%}")
        report.append(f"- Annualized Return: {performance.annual_return:.2%}")
        report.append(f"- Sharpe Ratio: {performance.sharpe_ratio:.3f}")
        report.append(f"- Maximum Drawdown: {performance.max_drawdown:.2%}")
        
        report.append("\n## Trade Analysis")
        report.append(f"- Average Trade: ${performance.avg_trade:.2f}")
        report.append(f"- Average Win: ${performance.avg_win:.2f}")
        report.append(f"- Average Loss: ${performance.avg_loss:.2f}")
        report.append(f"- Largest Win: ${performance.largest_win:.2f}")
        report.append(f"- Largest Loss: ${performance.largest_loss:.2f}")
        report.append(f"- Profit Factor: {performance.profit_factor:.3f}")
        
        return "\n".join(report)

# Initialize backtester
backtester = TradingStrategyBacktester(signal_generator, TRADING_CONFIG)
print("✅ TradingStrategyBacktester initialized")

✅ TradingStrategyBacktester initialized


In [7]:
# Usage Examples and Integration Workflow
print("🚀 Trading Strategy Integration System Ready!")
print("\nNext steps to use hyperparameter optimizations in trading:")
print("\n1️⃣  LOAD OPTIMIZED PARAMETERS")
print("2️⃣  BUILD AND TRAIN MODELS")
print("3️⃣  GENERATE TRADING SIGNALS")
print("4️⃣  BACKTEST STRATEGIES")
print("5️⃣  DEPLOY FOR LIVE TRADING")

# Example 1: Load optimized parameters and build model
def step1_load_optimized_model(symbol: str = 'EURUSD'):
    """Step 1: Load optimized parameters and build model"""
    print(f"\n🎯 Step 1: Loading optimized model for {symbol}...")
    
    # Load best parameters from optimization results
    params_data = model_loader.load_best_parameters(symbol)
    
    if params_data:
        print(f"✅ Found optimization results for {symbol}")
        
        # Build model with optimized hyperparameters
        # Note: Input shape will depend on your feature engineering
        # This is just an example - adjust based on your actual features
        input_shape = (20, 50)  # (lookback_window, num_features)
        
        model = model_loader.build_optimized_model(symbol, input_shape)
        
        if model:
            print(f"✅ Built optimized model for {symbol}")
            print(f"   Parameters: {model.count_params():,}")
            return model
        else:
            print(f"❌ Failed to build model for {symbol}")
    else:
        print(f"❌ No optimization results found for {symbol}")
        print("   Run hyperparameter optimization first!")
    
    return None

# Example 2: Train model with real data (placeholder)
def step2_train_model(symbol: str = 'EURUSD'):
    """Step 2: Train model with actual data"""
    print(f"\n🎯 Step 2: Training model for {symbol}...")
    
    # Load model
    model = model_loader.loaded_models.get(symbol)
    if model is None:
        print(f"❌ No model loaded for {symbol}. Run step 1 first.")
        return
    
    # Load and prepare data
    features = data_pipeline.create_trading_features(symbol)
    if features is None:
        print(f"❌ No data available for {symbol}")
        return
    
    print(f"✅ Loaded {features.shape[0]} data points with {features.shape[1]} features")
    print("\n⚠️  Training step is simplified for demonstration")
    print("   In production, you would:")
    print("   - Create proper train/validation/test splits")
    print("   - Generate target labels for price direction")
    print("   - Scale features appropriately")
    print("   - Train model with early stopping")
    print("   - Validate model performance")
    
    # Placeholder for actual training
    # model.fit(X_train, y_train, validation_data=(X_val, y_val), ...)
    
    # Save trained model (placeholder)
    # model_loader.save_trained_model(symbol, model, scaler)
    
    print(f"📝 Model training placeholder completed for {symbol}")

# Example 3: Generate trading signals
def step3_generate_signals(symbol: str = 'EURUSD'):
    """Step 3: Generate trading signals"""
    print(f"\n🎯 Step 3: Generating trading signals for {symbol}...")
    
    # Check if we have data
    price_data = data_pipeline.load_symbol_data(symbol)
    if price_data is None:
        print(f"❌ No price data for {symbol}")
        return
    
    # Get recent date for signal generation
    recent_date = price_data.index[-100]  # Use a date with enough lookback
    
    print(f"   Generating signal for {recent_date}")
    
    # Generate signal (this will use dummy model if no trained model exists)
    signal = signal_generator.generate_signals(
        symbol, recent_date, 
        confidence_threshold_high=0.6,
        confidence_threshold_low=0.4
    )
    
    if signal:
        print(f"✅ Generated signal for {symbol}:")
        print(f"   Signal: {signal.signal} ({'BUY' if signal.signal == 1 else 'SELL' if signal.signal == -1 else 'HOLD'})")
        print(f"   Confidence: {signal.confidence:.3f}")
        print(f"   Price: {signal.price:.5f}")
        print(f"   Stop Loss: {signal.stop_loss:.5f}" if signal.stop_loss else "   Stop Loss: None")
        print(f"   Take Profit: {signal.take_profit:.5f}" if signal.take_profit else "   Take Profit: None")
        return signal
    else:
        print(f"❌ Failed to generate signal for {symbol}")
        print("   This could be due to missing trained model or insufficient data")
    
    return None

# Example 4: Run backtest
def step4_run_backtest(symbol: str = 'EURUSD', days: int = 30):
    """Step 4: Run strategy backtest"""
    print(f"\n🎯 Step 4: Running backtest for {symbol}...")
    
    # Get data for backtest period
    price_data = data_pipeline.load_symbol_data(symbol)
    if price_data is None:
        print(f"❌ No price data for {symbol}")
        return
    
    # Define backtest period (last N days)
    end_date = price_data.index[-1]
    start_date = end_date - timedelta(days=days)
    
    print(f"   Backtest period: {start_date.date()} to {end_date.date()}")
    
    # Run backtest
    performance = backtester.run_backtest(
        symbol, start_date, end_date,
        confidence_threshold_high=0.6,
        confidence_threshold_low=0.4
    )
    
    if performance:
        # Generate performance report
        report = backtester.create_performance_report(symbol)
        print("\n" + "="*60)
        print(report)
        print("="*60)
        
        return performance
    else:
        print(f"❌ Backtest failed for {symbol}")
    
    return None

# Example 5: Complete workflow
def step5_complete_workflow(symbol: str = 'EURUSD'):
    """Step 5: Complete workflow demonstration"""
    print(f"\n🎯 Step 5: Complete workflow for {symbol}...")
    
    # Step 1: Load optimized model
    model = step1_load_optimized_model(symbol)
    
    # Step 2: Train model (placeholder)
    step2_train_model(symbol)
    
    # Step 3: Generate signals
    signal = step3_generate_signals(symbol)
    
    # Step 4: Run backtest
    performance = step4_run_backtest(symbol, days=30)
    
    print(f"\n🎉 Complete workflow demonstration finished for {symbol}!")
    
    return {
        'model': model,
        'signal': signal,
        'performance': performance
    }

print("\n💡 Usage:")
print("  - step1_load_optimized_model('EURUSD')  # Load optimized parameters")
print("  - step2_train_model('EURUSD')           # Train model with data")
print("  - step3_generate_signals('EURUSD')      # Generate trading signals")
print("  - step4_run_backtest('EURUSD', 30)      # Backtest strategy")
print("  - step5_complete_workflow('EURUSD')     # Run complete workflow")

print("\n🎉 Trading Strategy Integration System Ready!")
print(f"📁 Results will be saved to: {STRATEGIES_PATH}/")
print("🔧 Ready to convert optimizations to trading strategies!")

print("\n📋 Next Steps for Production:")
print("1. Run hyperparameter optimization (Advanced_Hyperparameter_Optimization_Clean.ipynb)")
print("2. Use this notebook to convert optimizations to trading strategies")
print("3. Implement real data pipeline for your data source")
print("4. Add proper model training with target generation")
print("5. Integrate with your trading platform for live execution")
print("6. Implement risk management and position sizing")
print("7. Set up monitoring and alerting systems")

🚀 Trading Strategy Integration System Ready!

Next steps to use hyperparameter optimizations in trading:

1️⃣  LOAD OPTIMIZED PARAMETERS
2️⃣  BUILD AND TRAIN MODELS
3️⃣  GENERATE TRADING SIGNALS
4️⃣  BACKTEST STRATEGIES
5️⃣  DEPLOY FOR LIVE TRADING

💡 Usage:
  - step1_load_optimized_model('EURUSD')  # Load optimized parameters
  - step2_train_model('EURUSD')           # Train model with data
  - step3_generate_signals('EURUSD')      # Generate trading signals
  - step4_run_backtest('EURUSD', 30)      # Backtest strategy
  - step5_complete_workflow('EURUSD')     # Run complete workflow

🎉 Trading Strategy Integration System Ready!
📁 Results will be saved to: trading_strategies/
🔧 Ready to convert optimizations to trading strategies!

📋 Next Steps for Production:
1. Run hyperparameter optimization (Advanced_Hyperparameter_Optimization_Clean.ipynb)
2. Use this notebook to convert optimizations to trading strategies
3. Implement real data pipeline for your data source
4. Add proper model t

In [9]:
#step1_load_optimized_model('EURUSD')  # Load optimized parameters
#step2_train_model('EURUSD')           # Train model with data
#step3_generate_signals('EURUSD')      # Generate trading signals
#step4_run_backtest('EURUSD', 30)      # Backtest strategy
step5_complete_workflow('EURUSD')     # Run complete workflow


🎯 Step 5: Complete workflow for EURUSD...

🎯 Step 1: Loading optimized model for EURUSD...
✅ Loaded parameters for EURUSD from best_params_EURUSD_20250612_225026.json
   Objective value: 0.890595
   Accuracy: 0.8015
✅ Found optimization results for EURUSD
✅ Loaded parameters for EURUSD from best_params_EURUSD_20250612_225026.json
   Objective value: 0.890595
   Accuracy: 0.8015
✅ Built optimized model for EURUSD
   Model parameters: 65,043
✅ Built optimized model for EURUSD
   Parameters: 65,043

🎯 Step 2: Training model for EURUSD...
✅ Loaded EURUSD: 5000 rows from metatrader_EURUSD.parquet
✅ Created 51 features for EURUSD
✅ Loaded 5000 data points with 51 features

⚠️  Training step is simplified for demonstration
   In production, you would:
   - Create proper train/validation/test splits
   - Generate target labels for price direction
   - Scale features appropriately
   - Train model with early stopping
   - Validate model performance
📝 Model training placeholder completed for EU

{'model': <Sequential name=sequential_1, built=True>,
 'signal': None,
 'performance': StrategyPerformance(symbol='EURUSD', total_trades=0, winning_trades=0, losing_trades=0, win_rate=0, total_return=0, annual_return=0, sharpe_ratio=0, max_drawdown=0, profit_factor=0, avg_trade=0, avg_win=0, avg_loss=0, largest_win=0, largest_loss=0)}